In [2]:
import numpy as np
import pickle

def load_cifar10_batch(file):
    with open(file, 'rb') as f:
        batch = pickle.load(f, encoding='latin1')
    return batch

def preprocess_cifar10_data(data):
    images = data['data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  
    labels = np.array(data['labels'])
    images = images.astype('float32') / 255.0
    
    return images, labels

def load_and_preprocess_cifar10(data_dir):
    train_images = []
    train_labels = []
    for i in range(1, 6):
        batch_file = f'{data_dir}/data_batch_{i}'
        batch = load_cifar10_batch(batch_file)
        images, labels = preprocess_cifar10_data(batch)
        train_images.append(images)
        train_labels.append(labels)
    
    train_images = np.concatenate(train_images, axis=0)
    train_labels = np.concatenate(train_labels, axis=0)
    test_batch = load_cifar10_batch(f'{data_dir}/test_batch')
    test_images, test_labels = preprocess_cifar10_data(test_batch)
    return (train_images, train_labels), (test_images, test_labels)

data_dir = 'cifar-10'
(train_images, train_labels), (test_images, test_labels) = load_and_preprocess_cifar10(data_dir)

print("训练图像形状:", train_images.shape)
print("训练标签形状:", train_labels.shape)
print("测试图像形状:", test_images.shape)
print("测试标签形状:", test_labels.shape)

训练图像形状: (50000, 32, 32, 3)
训练标签形状: (50000,)
测试图像形状: (10000, 32, 32, 3)
测试标签形状: (10000,)


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_encoder(input_shape=(32, 32, 3)):
    encoder = models.Sequential(name="Encoder")
    encoder.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    encoder.add(layers.MaxPooling2D((2, 2), strides=2))  
    encoder.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    encoder.add(layers.MaxPooling2D((2, 2), strides=2)) 
    encoder.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    encoder.add(layers.MaxPooling2D((2, 2), strides=2))  
    return encoder

encoder = build_encoder()
encoder.summary()

D:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "Encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 93,248 (364.25 KB)

 Trainable params: 93,248 (364.25 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
from tensorflow.keras import layers, models

def build_decoder(latent_shape=(4, 4, 128)):
    decoder = models.Sequential(name="Decoder")
    decoder.add(layers.Conv2DTranspose(128, (3, 3), strides=2, padding='same', activation='relu', input_shape=latent_shape))  
    decoder.add(layers.Conv2DTranspose(64, (3, 3), strides=2, padding='same', activation='relu'))  
    decoder.add(layers.Conv2DTranspose(32, (3, 3), strides=2, padding='same', activation='relu')) 
    decoder.add(layers.Conv2D(3, (3, 3), padding='same', activation='sigmoid')) 
    return decoder

def build_classifier(latent_shape=(4, 4, 128), num_classes=10):
    classifier = models.Sequential(name="Classifier")
    classifier.add(layers.Flatten(input_shape=latent_shape))  
    classifier.add(layers.Dense(256, activation='relu'))  
    classifier.add(layers.Dropout(0.5))  
    classifier.add(layers.Dense(num_classes, activation='softmax'))  
    return classifier
decoder = build_decoder()
classifier = build_classifier()
print("Decoder Summary:")
decoder.summary()

print("\nClassifier Summary:")
classifier.summary()

Decoder Summary:


D:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv_transpose.py:94: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
D:\anaconda\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "Decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_transpose                │ (None, 8, 8, 128)      │       147,584 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 16, 16, 64)     │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 32, 32, 32)     │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 3)      │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 240,707 (940.26 KB)

...

Model: "Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 527,114 (2.01 MB)

 Trainable params: 527,114 (2.01 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers
alpha = 1.0  
beta = 1.0   

def build_full_model(encoder, decoder, classifier):
    input_image = layers.Input(shape=(32, 32, 3), name="input_image")

    latent_features = encoder(input_image)

    reconstructed_image = decoder(latent_features)

    class_probabilities = classifier(latent_features)

    full_model = models.Model(inputs=input_image, outputs=[reconstructed_image, class_probabilities], name="Full_Model")
    return full_model

full_model = build_full_model(encoder, decoder, classifier)
full_model.summary()

reconstruction_loss_fn = losses.MeanSquaredError()  
classification_loss_fn = losses.SparseCategoricalCrossentropy()  
optimizer = optimizers.Adam(learning_rate=0.001)

full_model.compile(optimizer=optimizer,
                   loss={'Decoder': reconstruction_loss_fn, 'Classifier': classification_loss_fn},
                   loss_weights={'Decoder': alpha, 'Classifier': beta})
def train_step(model, x_batch, y_batch, alpha, beta):
    with tf.GradientTape() as tape:
        reconstructed_images, class_probabilities = model(x_batch)

        reconstruction_loss = reconstruction_loss_fn(x_batch, reconstructed_images)

        classification_loss = classification_loss_fn(y_batch, class_probabilities)

        total_loss = alpha * reconstruction_loss + beta * classification_loss

    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return total_loss, reconstruction_loss, classification_loss

def train_model(model, train_images, train_labels, epochs=10, batch_size=64):
    num_samples = train_images.shape[0]
    num_batches = num_samples // batch_size

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        epoch_total_loss = 0.0
        epoch_reconstruction_loss = 0.0
        epoch_classification_loss = 0.0
        
        indices = tf.random.shuffle(tf.range(num_samples))
        train_images = tf.gather(train_images, indices)
        train_labels = tf.gather(train_labels, indices)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size
            x_batch = train_images[start_idx:end_idx]
            y_batch = train_labels[start_idx:end_idx]
            total_loss, reconstruction_loss, classification_loss = train_step(model, x_batch, y_batch, alpha, beta)

            epoch_total_loss += total_loss
            epoch_reconstruction_loss += reconstruction_loss
            epoch_classification_loss += classification_loss
            if (batch_idx + 1) % 100 == 0:
                print(f"Batch {batch_idx + 1}/{num_batches} - "
                      f"Total Loss: {total_loss:.4f}, "
                      f"Reconstruction Loss: {reconstruction_loss:.4f}, "
                      f"Classification Loss: {classification_loss:.4f}")

        print(f"Epoch {epoch + 1} - "
              f"Avg Total Loss: {epoch_total_loss / num_batches:.4f}, "
              f"Avg Reconstruction Loss: {epoch_reconstruction_loss / num_batches:.4f}, "
              f"Avg Classification Loss: {epoch_classification_loss / num_batches:.4f}")
        print("--------------------------------------------------")

train_model(full_model, train_images, train_labels, epochs=10, batch_size=64)

Model: "Full_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Encoder             │ (None, 4, 4, 128) │     93,248 │ input_image[0][0] │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Decoder             │ (None, 32, 32, 3) │    240,707 │ Encoder[0][0]     │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Classifier          │ (None, 10)        │    527,114 │ Encoder[0][0]     │
│ (Sequential)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 861,069 (3.28 MB)

 Trainable params: 861,069 (3.28 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
Batch 100/781 - Total Loss: 1.8556, Reconstruction Loss: 0.0325, Classification Loss: 1.8230
Batch 200/781 - Total Loss: 1.3355, Reconstruction Loss: 0.0214, Classification Loss: 1.3142
Batch 300/781 - Total Loss: 1.3421, Reconstruction Loss: 0.0205, Classification Loss: 1.3216
Batch 400/781 - Total Loss: 1.2402, Reconstruction Loss: 0.0181, Classification Loss: 1.2221
Batch 500/781 - Total Loss: 1.3494, Reconstruction Loss: 0.0190, Classification Loss: 1.3304
Batch 600/781 - Total Loss: 1.0633, Reconstruction Loss: 0.0170, Classification Loss: 1.0463
Batch 700/781 - Total Loss: 1.2738, Reconstruction Loss: 0.0146, Classification Loss: 1.2592
Epoch 1 - Avg Total Loss: 1.4325, Avg Reconstruction Loss: 0.0228, Avg Classification Loss: 1.4097
--------------------------------------------------
Epoch 2/10
Batch 100/781 - Total Loss: 0.8980, Reconstruction Loss: 0.0188, Classification Loss: 0.8792
Batch 200/781 - Total Loss: 0.9202, Reconstruction Loss: 0.0142, Classification Loss

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers, metrics
alpha = 1.0  
beta = 1.0   

def build_full_model(encoder, decoder, classifier):
    input_image = layers.Input(shape=(32, 32, 3), name="input_image")
    latent_features = encoder(input_image)
    reconstructed_image = decoder(latent_features)
    class_probabilities = classifier(latent_features)
    full_model = models.Model(inputs=input_image, outputs=[reconstructed_image, class_probabilities], name="Full_Model")
    return full_model

full_model = build_full_model(encoder, decoder, classifier)

reconstruction_loss_fn = losses.MeanSquaredError()  
classification_loss_fn = losses.SparseCategoricalCrossentropy()  

optimizer = optimizers.Adam(learning_rate=0.001)

def train_step(model, x_batch, y_batch, alpha, beta):
    with tf.GradientTape() as tape:
        reconstructed_images, class_probabilities = model(x_batch)

        reconstruction_loss = reconstruction_loss_fn(x_batch, reconstructed_images)

        classification_loss = classification_loss_fn(y_batch, class_probabilities)

        total_loss = alpha * reconstruction_loss + beta * classification_loss

    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return total_loss, reconstruction_loss, classification_loss

def train_model(model, train_images, train_labels, test_images, test_labels, epochs=10, batch_size=64):
    num_samples = train_images.shape[0]
    num_batches = num_samples // batch_size

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        epoch_total_loss = 0.0
        epoch_reconstruction_loss = 0.0
        epoch_classification_loss = 0.0

        indices = tf.random.shuffle(tf.range(num_samples))
        train_images = tf.gather(train_images, indices)
        train_labels = tf.gather(train_labels, indices)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size
            x_batch = train_images[start_idx:end_idx]
            y_batch = train_labels[start_idx:end_idx]
            total_loss, reconstruction_loss, classification_loss = train_step(model, x_batch, y_batch, alpha, beta)
            epoch_total_loss += total_loss
            epoch_reconstruction_loss += reconstruction_loss
            epoch_classification_loss += classification_loss

            if (batch_idx + 1) % 100 == 0:
                print(f"Batch {batch_idx + 1}/{num_batches} - "
                      f"Total Loss: {total_loss:.4f}, "
                      f"Reconstruction Loss: {reconstruction_loss:.4f}, "
                      f"Classification Loss: {classification_loss:.4f}")
        print(f"Epoch {epoch + 1} - "
              f"Avg Total Loss: {epoch_total_loss / num_batches:.4f}, "
              f"Avg Reconstruction Loss: {epoch_reconstruction_loss / num_batches:.4f}, "
              f"Avg Classification Loss: {epoch_classification_loss / num_batches:.4f}")
        evaluate_model(model, test_images, test_labels)
        print("--------------------------------------------------")

def evaluate_model(model, test_images, test_labels):
    reconstructed_images, class_probabilities = model(test_images)

    reconstruction_loss = reconstruction_loss_fn(test_images, reconstructed_images).numpy()

    predicted_labels = tf.argmax(class_probabilities, axis=1)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted_labels, test_labels), tf.float32)).numpy()

    print(f"Test Set Evaluation - "
          f"Reconstruction Loss (MSE): {reconstruction_loss:.4f}, "
          f"Classification Accuracy: {accuracy * 100:.2f}%")
train_model(full_model, train_images, train_labels, test_images, test_labels, epochs=10, batch_size=64) 

Epoch 1/10
Batch 100/781 - Total Loss: 0.1199, Reconstruction Loss: 0.0107, Classification Loss: 0.1093
Batch 200/781 - Total Loss: 0.2053, Reconstruction Loss: 0.0105, Classification Loss: 0.1949
Batch 300/781 - Total Loss: 0.2133, Reconstruction Loss: 0.0115, Classification Loss: 0.2018
Batch 400/781 - Total Loss: 0.1784, Reconstruction Loss: 0.0109, Classification Loss: 0.1675
Batch 500/781 - Total Loss: 0.2704, Reconstruction Loss: 0.0122, Classification Loss: 0.2583
Batch 600/781 - Total Loss: 0.3599, Reconstruction Loss: 0.0114, Classification Loss: 0.3486
Batch 700/781 - Total Loss: 0.1128, Reconstruction Loss: 0.0118, Classification Loss: 0.1010
Epoch 1 - Avg Total Loss: 0.2109, Avg Reconstruction Loss: 0.0118, Avg Classification Loss: 0.1990
Test Set Evaluation - Reconstruction Loss (MSE): 0.0120, Classification Accuracy: 74.01%
--------------------------------------------------
Epoch 2/10
Batch 100/781 - Total Loss: 0.1272, Reconstruction Loss: 0.0109, Classification Loss: 0.

In [7]:
def evaluate_model(model, test_images, test_labels):
    reconstructed_images, class_probabilities = model(test_images)
    reconstruction_loss = reconstruction_loss_fn(test_images, reconstructed_images).numpy()
    predicted_labels = tf.argmax(class_probabilities, axis=1)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted_labels, test_labels), tf.float32)).numpy()
    print(f"Test Set Evaluation - "
          f"Reconstruction Loss (MSE): {reconstruction_loss:.4f}, "
          f"Classification Accuracy: {accuracy * 100:.2f}%")
    return reconstruction_loss, accuracy
test_reconstruction_loss, test_accuracy = evaluate_model(full_model, test_images, test_labels)

Test Set Evaluation - Reconstruction Loss (MSE): 0.0114, Classification Accuracy: 74.10%
